In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from altaipony.lcio import from_path

from error import refine_detrended_flux_err

In [3]:
df = pd.read_csv("../k2scoc/results/tables/improved_flares_vetted.csv")
#df = df[(df.real == 1) & (df.note.isnull())]
df.head()

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,EPIC,C,real,length,note,cluster
0,2005,2030,105828,105854,2271.380070,2271.911291,390.877871,4.193384,0.016548,3282,210317378,4,0.0,25,NaN,hyades
1,1907,1935,105729,105758,2269.357343,2269.949859,284.387645,4.848855,0.009683,3282,210317378,4,0.0,28,NaN,hyades
2,1870,1876,105691,105698,2268.580942,2268.723963,73.404611,7.026491,0.006631,3282,210317378,4,0.0,6,NaN,hyades
3,1831,1841,105651,105662,2267.763678,2267.988426,96.207856,6.176100,0.006221,3282,210317378,4,0.0,10,NaN,hyades
4,1171,1174,104966,104969,2253.767988,2253.829284,27.441864,3.044179,0.005928,3282,210317378,4,1.0,3,NaN,hyades


In [4]:
luminosities = pd.DataFrame()
for cl in ["ngc2682","ngc6774","praesepe","pleiades","hyades"]:
    lum = pd.read_csv(f"../StellarParameters/luminosities/{cl}_luminosities.csv")
    lum = lum[["EPIC","Lum_Kepler", "e_Lum_Kepler", "Teff_median","Teff_std", "todrop"]]
    lum.EPIC = lum.EPIC.astype(int)
    lum = lum.drop_duplicates()
    luminosities = luminosities.append(lum)

#df = df[(df.real == 1) & (df.note.isnull())]
print(luminosities.shape, luminosities.drop_duplicates().shape)
luminosities.head()

(3223, 6) (3223, 6)


,EPIC,Lum_Kepler,e_Lum_Kepler,Teff_median,Teff_std,todrop
0,211389428,1.952370e+31,3.641215e+30,4065.686917,86.142585,NaN
1,211389537,6.497467e+31,8.005714e+30,4859.417724,114.038717,g-z outlier
2,211389716,3.876991e+32,1.050844e+32,5995.022873,278.839086,NaN
3,211389931,2.112355e+31,3.856409e+30,4117.019702,121.777132,NaN
4,211390158,5.477966e+32,1.037648e+32,6271.767690,250.289954,NaN


## What are the lightcurves without flares?

take full flares table 

In [5]:
clus = pd.read_csv("joblistonlyk2sc.sh",delimiter=" ",
                 names = ["python","call","EPIC","channel","C","cluster"
                         ])

In [6]:
df1 = clus[["EPIC","C"]]
df2 = df[["EPIC","C"]].drop_duplicates()

df3 = pd.concat([df1,df2]).drop_duplicates(keep=False)
df3.shape
df3.head()

,EPIC,C
0,211389428,18
1,211389537,5
3,211389931,18
4,211390158,5
5,211391125,5


In [7]:
for label, r in df3.iterrows():
    EPIC, C = int(r.EPIC), int(r.C)
    try:
        flc = from_path(f"../k2scoc/pony_k2sc_k2_llc_{EPIC}-c{C:02d}_kepler_v2_lc.fits",
                            mode="AltaiPony", mission="K2")
        flc = refine_detrended_flux_err(flc)

        df3.loc[(df3.EPIC == EPIC) & (df3.C == C),"dtp_tot"] = flc.flux.shape[0]
    except:
        df3.loc[(df3.EPIC == EPIC) & (df3.C == C),"dtp_tot"] = 0.

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211389428-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/ekaterina/Documents/001_science/flares-in-clusters-with-k2-ii/VettingFlares/error.py:42: RuntimeWarning: invalid value encountered in greater
  filtered[flcd.detrended_flux - flcd.it_med > mask_pos_outliers_sigma * flcd.detrended_flux_err] = np.nan
/home/ekaterina/Documents/001_science/flares-in-clusters-with-k2-ii/VettingFlare

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211395891-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211396155-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211396551-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211404466-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_200233440-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_200233433-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_200233303-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211417559-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211417817-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211430257-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211430472-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211418141-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

...

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211892153-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211919747-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211818650-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211835162-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212034762-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212112321-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211918097-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211891321-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211988080-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211970861-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_219437560-c07_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_219597943-c07_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_219462497-c07_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_219297228-c07_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_219353203-c07_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

In [8]:
df3b = df3.merge(clus[["C","EPIC","cluster"]], how="left", on=["C","EPIC"])

In [9]:
dffinal = pd.concat([df,df3b])
dffinal.tail()

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,EPIC,C,real,length,note,cluster
867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3449.0,219210446,7,NaN,NaN,NaN,ngc6774
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3449.0,219280168,7,NaN,NaN,NaN,ngc6774
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3449.0,219037489,7,NaN,NaN,NaN,ngc6774
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3449.0,219556441,7,NaN,NaN,NaN,ngc6774
871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3449.0,219439085,7,NaN,NaN,NaN,ngc6774


In [10]:
dffinal.shape

(6354, 16)

## What are cluster-wise observing times?

In [11]:
dffinal.loc[dffinal.cluster == "ngc6774", "dtp_tot"].sum() / 2. / 24. / 365.25

35.017225644535706

## Merge with luminosities

In [12]:
dflum = dffinal.merge(luminosities,how="left", on="EPIC")
dflum.head()

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,...,C,real,length,note,cluster,Lum_Kepler,e_Lum_Kepler,Teff_median,Teff_std,todrop
0,2005.0,2030.0,105828.0,105854.0,2271.380070,2271.911291,390.877871,4.193384,0.016548,3282.0,...,4,0.0,25.0,NaN,hyades,3.562501e+30,9.535661e+29,3511.307394,75.080192,NaN
1,1907.0,1935.0,105729.0,105758.0,2269.357343,2269.949859,284.387645,4.848855,0.009683,3282.0,...,4,0.0,28.0,NaN,hyades,3.562501e+30,9.535661e+29,3511.307394,75.080192,NaN
2,1870.0,1876.0,105691.0,105698.0,2268.580942,2268.723963,73.404611,7.026491,0.006631,3282.0,...,4,0.0,6.0,NaN,hyades,3.562501e+30,9.535661e+29,3511.307394,75.080192,NaN
3,1831.0,1841.0,105651.0,105662.0,2267.763678,2267.988426,96.207856,6.176100,0.006221,3282.0,...,4,0.0,10.0,NaN,hyades,3.562501e+30,9.535661e+29,3511.307394,75.080192,NaN
4,1171.0,1174.0,104966.0,104969.0,2253.767988,2253.829284,27.441864,3.044179,0.005928,3282.0,...,4,1.0,3.0,NaN,hyades,3.562501e+30,9.535661e+29,3511.307394,75.080192,NaN


In [13]:
dflum.shape[0], df.shape[0] + df3b.shape[0]

(6354, 6354)

In [14]:
dflum.to_csv("../k2scoc/results/tables/full_table.csv",index=False)